# Checks if Perfsonar data is indexed

This notebook checks number of indexed documents in all four doc types (latency, packetloss, throughput and traceroute) and alerts if any of them is 0. It sends mails to all the people substribed to that alert. It is run every 30 min from a cron job.

In [18]:
import subscribers, alerts
from datetime import datetime, timedelta
from elasticsearch import Elasticsearch, exceptions as es_exceptions
es = Elasticsearch(hosts=[{'host':'atlas-kibana.mwt2.org', 'port':9200}],timeout=60)

## find all the indices to look at

In [20]:
indices = es.cat.indices(index="network_weather_2-*", h="index", request_timeout=600).split('\n')
indices = [x for x in indices if x != '']
indices = [x.strip() for x in indices]
#print(indices)

cday = datetime.utcnow()
pday = cday - timedelta(days=1)
ind1 = 'network_weather_2-%d.%d.%d' % (cday.year, cday.month, cday.day)
ind2 = 'network_weather_2-%d.%d.%d' % (pday.year, pday.month, pday.day)

print('checking for indices:', ind1, ind2)

ind=[]
if ind1 in indices:
    ind.append(ind1)
if ind2 != ind1 and ind2 in indices and cday.hour<9:   # not necessarily 3, just indicate it is the beginning period of new day
    ind.append(ind2)

if len(ind)==0:
    print('no current indices found. Aborting.')
    exit
else:
    print('will use indices:', ind)

print('current time',datetime.utcnow())

checking for indices: network_weather_2-2016.8.22 network_weather_2-2016.8.21
will use indices: ['network_weather_2-2016.8.22']
current time 2016-08-22 21:11:13.103942


In [21]:
# There is a time offset here - we do now-9 instead of expected now-1. There are two possible reasons:
# a) timestamps are correct but we simply get data with a long delay
# b) timestamps are wrong.

types_query={
   "size": 0,
   "query": {
        "bool":{
            "must": {
#                "term": {"_type":"latency"}
            },
            "filter":{
                "range":{"timestamp":{"gt": "now-9h"}}
            }
        }
   },
    "aggs" : {
        "docType" : {
            "terms" : { "field" : "_type" }
        }
    }
}

res = es.search(index=ind, body=types_query, request_timeout=120)
print(res)

{'timed_out': False, '_shards': {'total': 5, 'successful': 5, 'failed': 0}, 'aggregations': {'docType': {'doc_count_error_upper_bound': 0, 'buckets': [{'key': 'latency', 'doc_count': 200}, {'key': 'packet_loss_rate', 'doc_count': 199}, {'key': 'traceroute', 'doc_count': 56}], 'sum_other_doc_count': 0}}, 'hits': {'total': 455, 'hits': [], 'max_score': 0.0}, 'took': 24}


In [22]:
problematic=[]

if res['hits']['total']==0:
    problematic=['latency', 'packet_loss_rate', 'traceroute', 'throughput']

r=res['aggregations']['docType']['buckets']
for t in r:
    print(t)
    if t['doc_count']==0:
        problematic.append(t['key'])
print('types with no docs:', problematic)

{'key': 'latency', 'doc_count': 200}
{'key': 'packet_loss_rate', 'doc_count': 199}
{'key': 'traceroute', 'doc_count': 56}
types with no docs: []


In [23]:
if len(problematic) > 0:
    S = subscribers.subscribers()
    A = alerts.alerts()

    testName = 'Perfsonar'
    subscribersToPerfsonar =  S.getSubscribers(testName)
    print (subscribersToPerfsonar)
    for subscriber in subscribersToPerfsonar:
        body = 'Dear ' + subscriber[0]+',\n\n'
        body = body + '\tthis mail is to let you that there is an issue in indexing Perfsonar data in UC Elasticsearch.\n'
        body = body + 'This is number of documents indexed in last 30 minutes per document type:\n'
        body = body + str(r)+'\n'
        body = body + '\nBest regards,\nATLAS AAS'
        body = body + '\n\n To change your alerts preferences please you the following link:\n'+subscriber[2]
        print(subscriber)
        A.sendMail(testName, subscriber[1], body)
        A.addAlert(testName, subscriber[0],str(r))